# Evaluation of experiments

taken and adapted from https://github.com/lukaskirchdorfer/AgentSimulator/blob/main/evaluation.ipynb


In [1]:
import os
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.log.util import sorting
from scipy.stats import wasserstein_distance
from utils import *

from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance

import warnings
warnings.filterwarnings("ignore")

In [2]:
path_sim_dir = os.path.join('..', 'data', 'simulated_data', 'threats')

## Evaluation function

In [3]:
def main_(log_paths, name_experiments):
    def perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log):
        for i in range(10):

            try: 
                # print(f"Evaluate simulation {i}")
                path_simulated_file = PATH_SIMULATED_LOG + '/simulated_log_' + str(i) + '.csv'
                # read simulated log and align column names
                simulated_log = pd.read_csv(path_simulated_file)
                simulated_log = align_column_names(simulated_log)
                # print(simulated_log)
                # print("########")
                # print(simulated_log[event_log_ids.activity].unique())
                simulated_log[event_log_ids.start_time] = pd.to_datetime(simulated_log[event_log_ids.start_time], utc=True, format='mixed')
                simulated_log[event_log_ids.end_time] = pd.to_datetime(simulated_log[event_log_ids.end_time], utc=True, format='mixed')

                # Call passing the event logs, and its column ID mappings
                ngd = n_gram_distribution_distance(test_log, event_log_ids, simulated_log, event_log_ids, n=3)
                all_metrics['NGD'].append(ngd)

                cadd = case_arrival_distribution_distance(
                    test_log, event_log_ids,  
                    simulated_log, event_log_ids,  
                )
                all_metrics['CADD'].append(cadd)

                # Call passing the event logs, its column ID mappings, timestamp type, and discretize function
                aedd = absolute_event_distribution_distance(
                    test_log, event_log_ids,  # First event log and its column id mappings
                    simulated_log, event_log_ids,  # Second event log and its column id mappings
                    discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                    discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
                )
                all_metrics['AEDD'].append(aedd)

                cedd = circadian_event_distribution_distance(
                    test_log, event_log_ids,  # First event log and its column id mappings
                    simulated_log, event_log_ids,  # Second event log and its column id mappings
                    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
                )
                all_metrics['CEDD'].append(cedd)

                redd = relative_event_distribution_distance(
                    test_log, event_log_ids,  # First event log and its column id mappings
                    simulated_log, event_log_ids,  # Second event log and its column id mappings
                    discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
                    discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
                )
                all_metrics['REDD'].append(redd)

                ctdd = cycle_time_distribution_distance(
                    test_log, event_log_ids,  # First event log and its column id mappings
                    simulated_log, event_log_ids,  # Second event log and its column id mappings
                    bin_size=pd.Timedelta(hours=1)  # Bins of 1 hour
                )
                all_metrics['CTDD'].append(ctdd)

                wipd = work_in_progress_distance(
                    test_log, event_log_ids,  
                    simulated_log, event_log_ids,  
                    Timedelta= pd.Timedelta(days=1)
                )
                all_metrics['WIPD'].append(wipd)
                
                work_in_progress_distance()

            except:
                pass

        return all_metrics
    
    number_evaluations = len(log_paths)

    # Set event log column ID mapping
    event_log_ids = EventLogIDs(  # These values are stored in DEFAULT_CSV_IDS
        case="case_id",
        activity="activity",
        start_time="start_time",
        end_time="end_time",
        resource='resource'
    )

    index_names = name_experiments
    results_df = pd.DataFrame(index=index_names)
    mean_results = pd.DataFrame(index=index_names)

    for experiment in range(number_evaluations):
        # Read and transform time attributes
        test_log = pd.read_csv(log_paths[experiment][0])
        # print(log_paths[experiment][0])
        # print(test_log.columns)
        if 'time:timestamp' in test_log.columns:
            test_log = test_log.drop(columns=["time:timestamp"])
        
        test_log = align_column_names(test_log)
        # print(test_log.columns)
        test_log[event_log_ids.start_time] = pd.to_datetime(test_log[event_log_ids.start_time], utc=True, format='mixed')
        test_log[event_log_ids.end_time]   = pd.to_datetime(test_log[event_log_ids.end_time],   utc=True, format='mixed')

        PATH_SIMULATED_LOG = log_paths[experiment][1]

        all_metrics = {
            'NGD': [],
            'CADD': [],
            'AEDD': [],
            'CEDD': [],
            'REDD': [],
            'CTDD': [],
            'WIPD': []
        }

        all_metrics = perform_evauluation(all_metrics, PATH_SIMULATED_LOG, test_log)

        mean_results.loc[index_names[experiment], 'N-Gram Distribution Distance'] = round(np.mean(all_metrics['NGD']), 3)
        mean_results.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = round(np.mean(all_metrics['CADD']), 3)
        mean_results.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = round(np.mean(all_metrics['AEDD']), 3)
        mean_results.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = round(np.mean(all_metrics['CEDD']), 3)
        mean_results.loc[index_names[experiment], 'Relative Event Distribution Distance'] = round(np.mean(all_metrics['REDD']), 3)
        mean_results.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = round(np.mean(all_metrics['CTDD']), 3)
        # mean_results.loc[index_names[experiment], 'Work In Progress Distribution Distance'] = round(np.mean(all_metrics['WIPD']), 3)


    
        results_df.loc[index_names[experiment], 'N-Gram Distribution Distance'] = f"{round(np.mean(all_metrics['NGD']), 3)} ({round(np.std(all_metrics['NGD']), 3)})"
        results_df.loc[index_names[experiment], 'Case Arrival Distribution Distance'] = f"{round(np.mean(all_metrics['CADD']), 3)} ({round(np.std(all_metrics['CADD']), 3)})"
        results_df.loc[index_names[experiment], 'Absolute Event Distribution Distance'] = f"{round(np.mean(all_metrics['AEDD']), 3)} ({round(np.std(all_metrics['AEDD']), 3)})"
        results_df.loc[index_names[experiment], 'Circadian Event Distribution Distance'] = f"{round(np.mean(all_metrics['CEDD']), 3)} ({round(np.std(all_metrics['CEDD']), 3)})"
        results_df.loc[index_names[experiment], 'Relative Event Distribution Distance'] = f"{round(np.mean(all_metrics['REDD']), 3)} ({round(np.std(all_metrics['REDD']), 3)})"
        results_df.loc[index_names[experiment], 'Cycle Time Distribution Distance'] = f"{round(np.mean(all_metrics['CTDD']), 3)} ({round(np.std(all_metrics['CTDD']), 3)})"
        # results_df.loc[index_names[experiment], 'Work In Progres Distribution Distance'] = f"{round(np.mean(all_metrics['CTDD']), 3)} ({round(np.std(all_metrics['WIPD']), 3)})"

    return mean_results, results_df
        
        

## T1: Concept drift

### Set up

In [4]:
logs = [
        'helpdesk_h1',   # scenario pre-drift
        'helpdesk_h1_2', # scenario across drift
    ]
split_dirs = [
    'helpdesk_two_scenarios'
]


In [5]:

def display_results(logs, split_dir):
    path_test1 = os.path.join(path_sim_dir, f'{split_dir}/{logs[0]}/main_results/test_preprocessed.csv')
    path_sim1  = os.path.join(path_sim_dir, f'{split_dir}/{logs[0]}/main_results')

    path_test2 = os.path.join(path_sim_dir, f'{split_dir}/{logs[1]}/main_results/test_preprocessed.csv')
    path_sim2  = os.path.join(path_sim_dir, f'{split_dir}/{logs[1]}/main_results')


    log_paths = [[path_test1, path_sim1], [path_test2, path_sim2]]
    mean_results, results_df = main_(log_paths, logs)

    display(results_df)


### Results

In [6]:

for split_dir in split_dirs:
    print(split_dir)
    display_results(logs, split_dir)

helpdesk_two_scenarios


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
helpdesk_h1,0.123 (0.007),428.012 (0.366),457.852 (4.839),1.031 (0.076),96.565 (1.507),234.367 (6.576)
helpdesk_h1_2,0.235 (0.006),3878.189 (47.478),4140.081 (42.678),1.151 (0.069),230.009 (0.958),225.803 (12.968)


## T2: Fading-in and out Phases in Extracted Logs
## &
## T3: Fading-out and in Phases around the Splitting Point

### Set up

In [7]:
output_target_dir = os.path.join(path_sim_dir, 't2_3')
split_dir = 't2_3'

# Define extraction types and split types
extraction_types = ['starting', 'contained', 'cut']
split_types = ['regular', 'intermediate', 'strict']

os.listdir(output_target_dir)

['BPIC_2012_W_cut_regular',
 'BPIC_2012_W_starting_regular',
 '.DS_Store',
 'BPIC_2012_W_starting_strict',
 'BPIC_2012_W_cut_strict',
 'BPIC_2012_W_cut_intermediate',
 'BPIC_2012_W_starting_intermediate',
 'BPIC_2012_W_contained_intermediate',
 'BPIC_2012_W_contained_regular',
 'BPIC_2012_W_contained_strict']

In [8]:
def generate_combinations(extraction_types, split_types, sorting_key):
    combinations = []
    
    if sorting_key == "extraction":
        for extraction in extraction_types:
            extraction_group = [[extraction, split] for split in split_types]
            combinations.append(extraction_group)
            
    elif sorting_key == "split":
        for split in split_types:
            split_group = [[extraction, split] for extraction in extraction_types]
            combinations.append(split_group)
            
    else:
        raise ValueError("Invalid sorting value. Use 'extraction' or 'split'.")
    
    return combinations


def get_log_list(log_name, file_names, extraction_types, split_types, sorting_key):
    # Prepare a nested list of matched files for each combination
    matched_files = []

    combinations = generate_combinations(extraction_types, split_types, sorting_key)

    for group in combinations:
        group_files = []
        for extraction, split in group:
            # Construct the pattern to match in the filenames
            pattern = f'{log_name}_{extraction}_{split}'
            # Find the corresponding file that matches the pattern
            matching_file = [file for file in file_names if file.startswith(pattern)]
            # Add the matched files directly (no list of length 1)
            group_files.extend(matching_file)
        matched_files.append(group_files)
    
    return matched_files


In [9]:

def display_results(logs, split_dir):
    path_test1 = os.path.join(path_sim_dir, f'{split_dir}/{logs[0]}/main_results/test_preprocessed.csv')
    path_sim1  = os.path.join(path_sim_dir, f'{split_dir}/{logs[0]}/main_results')

    path_test2 = os.path.join(path_sim_dir, f'{split_dir}/{logs[1]}/main_results/test_preprocessed.csv')
    path_sim2  = os.path.join(path_sim_dir, f'{split_dir}/{logs[1]}/main_results')

    path_test3 = os.path.join(path_sim_dir, f'{split_dir}/{logs[2]}/main_results/test_preprocessed.csv')
    path_sim3  = os.path.join(path_sim_dir, f'{split_dir}/{logs[2]}/main_results')

    log_paths = [[path_test1, path_sim1], [path_test2, path_sim2], [path_test3, path_sim3]]

    mean_results, results_df = main_(log_paths, logs)

    display(results_df)


In [10]:
log_name = 'BPIC_2012_W'

file_names = os.listdir(output_target_dir)


### Results

In [11]:
sorting_key = 'split'
matched_files = get_log_list(log_name, file_names, extraction_types, split_types, sorting_key)
for combination in matched_files:
    print(combination)
    display_results(combination, split_dir)

['BPIC_2012_W_starting_regular', 'BPIC_2012_W_contained_regular', 'BPIC_2012_W_cut_regular']


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
BPIC_2012_W_starting_regular,0.109 (0.011),22.313 (0.193),35.366 (6.033),4.587 (0.048),42.192 (5.194),71.16 (7.826)
BPIC_2012_W_contained_regular,0.236 (0.019),35.762 (0.158),95.337 (13.058),4.725 (0.029),81.294 (12.522),53.164 (8.399)
BPIC_2012_W_cut_regular,0.282 (0.01),26.117 (0.409),108.401 (11.379),4.346 (0.04),105.447 (11.019),60.503 (5.619)


['BPIC_2012_W_starting_intermediate', 'BPIC_2012_W_contained_intermediate', 'BPIC_2012_W_cut_intermediate']


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
BPIC_2012_W_starting_intermediate,0.165 (0.005),43.064 (0.165),42.695 (6.704),4.604 (0.04),73.207 (7.35),104.839 (4.02)
BPIC_2012_W_contained_intermediate,0.177 (0.017),27.425 (0.174),105.767 (10.401),4.784 (0.066),60.679 (7.724),33.017 (5.675)
BPIC_2012_W_cut_intermediate,0.262 (0.008),14.401 (0.319),118.541 (12.183),4.377 (0.034),83.244 (10.057),41.154 (3.608)


['BPIC_2012_W_starting_strict', 'BPIC_2012_W_contained_strict', 'BPIC_2012_W_cut_strict']


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
BPIC_2012_W_starting_strict,0.284 (0.008),163.415 (0.073),97.538 (6.429),4.705 (0.034),71.037 (5.929),103.664 (5.392)
BPIC_2012_W_contained_strict,0.431 (0.008),78.911 (0.0),25.182 (3.962),4.187 (0.103),95.225 (0.16),91.491 (0.047)
BPIC_2012_W_cut_strict,0.32 (0.012),127.873 (0.0),68.055 (6.109),3.964 (0.067),90.466 (0.115),107.673 (0.047)


In [12]:
sorting_key = 'extraction'
matched_files = get_log_list(log_name, file_names, extraction_types, split_types, sorting_key)
for combination in matched_files:
    print(combination)
    display_results(combination, split_dir)

['BPIC_2012_W_starting_regular', 'BPIC_2012_W_starting_intermediate', 'BPIC_2012_W_starting_strict']


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
BPIC_2012_W_starting_regular,0.109 (0.011),22.313 (0.193),35.366 (6.033),4.587 (0.048),42.192 (5.194),71.16 (7.826)
BPIC_2012_W_starting_intermediate,0.165 (0.005),43.064 (0.165),42.695 (6.704),4.604 (0.04),73.207 (7.35),104.839 (4.02)
BPIC_2012_W_starting_strict,0.284 (0.008),163.415 (0.073),97.538 (6.429),4.705 (0.034),71.037 (5.929),103.664 (5.392)


['BPIC_2012_W_contained_regular', 'BPIC_2012_W_contained_intermediate', 'BPIC_2012_W_contained_strict']


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
BPIC_2012_W_contained_regular,0.236 (0.019),35.762 (0.158),95.337 (13.058),4.725 (0.029),81.294 (12.522),53.164 (8.399)
BPIC_2012_W_contained_intermediate,0.177 (0.017),27.425 (0.174),105.767 (10.401),4.784 (0.066),60.679 (7.724),33.017 (5.675)
BPIC_2012_W_contained_strict,0.431 (0.008),78.911 (0.0),25.182 (3.962),4.187 (0.103),95.225 (0.16),91.491 (0.047)


['BPIC_2012_W_cut_regular', 'BPIC_2012_W_cut_intermediate', 'BPIC_2012_W_cut_strict']


,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
BPIC_2012_W_cut_regular,0.282 (0.01),26.117 (0.409),108.401 (11.379),4.346 (0.04),105.447 (11.019),60.503 (5.619)
BPIC_2012_W_cut_intermediate,0.262 (0.008),14.401 (0.319),118.541 (12.183),4.377 (0.034),83.244 (10.057),41.154 (3.608)
BPIC_2012_W_cut_strict,0.32 (0.012),127.873 (0.0),68.055 (6.109),3.964 (0.067),90.466 (0.115),107.673 (0.047)


## T5: Warm-up and Cool-down Phases of Simulated Logs

### Set up

In [13]:
logs = [
        'P2P'
    ]

In [14]:
def display_results(logs):
    for log in logs:
        PATH_TEST_LOG_MAS       = os.path.join(path_sim_dir, f'{log}/main_results/test_preprocessed.csv')
        PATH_SIMULATED_LOG_MAS  = os.path.join(path_sim_dir, f'{log}/main_results')

        PATH_TEST_WU      = os.path.join(path_sim_dir, f'{log}/warm_up/test_preprocessed.csv')
        PATH_SIMULATED_WU = os.path.join(path_sim_dir, f'{log}/warm_up')

        PATH_TEST_CD      = os.path.join(path_sim_dir, f'{log}/cool_down/test_preprocessed.csv')
        PATH_SIMULATED_CD = os.path.join(path_sim_dir, f'{log}/cool_down')

        PATH_TEST_WUCD      = os.path.join(path_sim_dir, f'{log}/warm_up_cool_down/test_preprocessed.csv')
        PATH_SIMULATED_WUCD = os.path.join(path_sim_dir, f'{log}/warm_up_cool_down')

        log_paths = [[PATH_TEST_LOG_MAS, PATH_SIMULATED_LOG_MAS], [PATH_TEST_WU, PATH_SIMULATED_WU], [PATH_TEST_CD, PATH_SIMULATED_CD],  [PATH_TEST_WUCD, PATH_SIMULATED_WUCD]]
        name_experiments = ['AgentSim', 'warm_up', 'cool_down', 'warm_up_cool_down']

        mean_results, results_df = main_(log_paths, name_experiments)

        styled_results_df = highlight_min_max_from_means(results_df, mean_results)

        display(log)
        display(styled_results_df)


### Results

In [15]:
display_results(logs)

'P2P'

,N-Gram Distribution Distance,Case Arrival Distribution Distance,Absolute Event Distribution Distance,Circadian Event Distribution Distance,Relative Event Distribution Distance,Cycle Time Distribution Distance
AgentSim,0.234 (0.018),863.101 (0.006),1275.176 (7.56),4.796 (0.195),766.631 (7.002),549.971 (13.588)
warm_up,0.228 (0.034),832.308 (0.003),1208.399 (14.04),5.162 (0.136),729.421 (13.986),487.861 (31.929)
cool_down,0.247 (0.021),824.249 (0.003),1167.954 (25.525),4.75 (0.257),698.485 (22.901),430.311 (46.867)
warm_up_cool_down,0.233 (0.021),836.832 (0.0),1116.946 (24.849),4.746 (0.17),635.676 (23.671),347.133 (38.979)
